In [1]:
#fast chekc to see if IP matches 
import requests
print(requests.get("https://api.ipify.org").text)

37.67.198.11


If the video has an available transcript, retrieves the full transcript text using youtube_transcript_api,
otherwise, returns the raised exception if no transcript available

In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs
import logging

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s [%(name)s] %(message)s",
    filename="yt_api.log",
    filemode="a",  # append
)

logger = logging.getLogger(__name__)

In [3]:
# video_url = "https://www.youtube.com/watch?v=GuM6dQGRFyQ"
# video_url = "https://www.youtube.com/watch?v=-hkEzwaY754"
# video_url = "https://www.youtube.com/watch?v=HhzpDM36Pzg"
video_url = "https://www.youtube.com/watch?v=pSbJ0aiMN0Y"

In [4]:
def get_youtube_video_id(url: str) -> str | None:
    parsed = urlparse(url)

    # youtu.be/<id>
    if parsed.netloc in ("youtu.be", "www.youtu.be"):
        return parsed.path.lstrip("/")

    # youtube.com/*
    if "youtube.com" in parsed.netloc:
        # watch?v=<id>
        if parsed.path == "/watch":
            return parse_qs(parsed.query).get("v", [None])[0]

        # /embed/<id>, /shorts/<id>
        parts = parsed.path.split("/")
        if len(parts) >= 3 and parts[1] in {"embed", "shorts"}:
            return parts[2]

    return None

def retrieve_video_transcript(video_url:str) -> str:
    video_id = get_youtube_video_id(video_url)
    ytt_api = YouTubeTranscriptApi()
    full_text=""
    try : 
        fetched_transcript = ytt_api.fetch(video_id)
    except Exception as e:
        logger.exception(
            "Failed to fetch transcript for video_url=%s",
            video_url,
        )
        return("")
    for snippet in fetched_transcript.snippets:
        full_text+= snippet.text + " "
    return(full_text)

In [5]:
test_text = retrieve_video_transcript(video_url)

In [6]:
test_text

''